In [25]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [26]:
# parametros cosecha
periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
lotes = []
penalizaciones = {
    'C1': [0.03, 0.02, 0.01, 0.005],
    'C2': [0.025, 0.012, 0.004, 0.002],
    'C3': [0.03, 0.02, 0.01, 0.005],
    'C4': [0.025, 0.012, 0.01, 0.002],
    'C5': [0.03, 0.02, 0.01, 0.005],
    'C6': [0.025, 0.012, 0.004, 0.002]
}

In [27]:
# Leo la base de datos y la guardo en una lista
c.execute("SELECT * FROM lotes")
lotes_obtenidos = c.fetchall()
lotes.append(lotes_obtenidos)
lotes = lotes[0]

In [28]:
#Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

# pasar lotes a lista de listas
lotes = [list(elem) for elem in lotes]
# Funcion que estima el dia de cosecha con media 0 y desv est 2
def dia_estimado():
        optimal_day = np.random.normal(0, 2)
        return int(round(optimal_day))
for lote in lotes:
    variacion_optimo = dia_estimado()
    lote[5]+=variacion_optimo
    lote.append(variacion_optimo)


In [29]:
def calcular_probabilidad_lluvia(P_seco_a_lluvioso, P_lluvioso_a_lluvioso):
    # Inicializar probabilidades iniciales según el estado inicial deseado
    P_dia_seco = 1
    P_dia_lluvioso = 0
    dias = 7

    # Calcular probabilidades de estado para cada día
    for _ in range(dias):
        P_dia_seco_nuevo = (P_dia_seco * (1 - P_seco_a_lluvioso) + P_dia_lluvioso * (1 - P_lluvioso_a_lluvioso))
        P_dia_lluvioso_nuevo = (P_dia_seco * P_seco_a_lluvioso + P_dia_lluvioso * ( P_lluvioso_a_lluvioso))
        P_dia_seco, P_dia_lluvioso = P_dia_seco_nuevo, P_dia_lluvioso_nuevo

    porcentaje_lluvia = P_dia_lluvioso 
    
    return porcentaje_lluvia

In [30]:
for lote in lotes:
    prob_lluvia = calcular_probabilidad_lluvia(lote[6],lote[7])
    prob_lluvia = round(prob_lluvia, 6)
    lote.append(prob_lluvia)

In [31]:
# Lotes actualizados
print(lotes)

[[1, 'L_1_C4', 1, 'C4', 217.5, 115, 0.43, 0.39, 0.257, -3, 0.413462], [2, 'L_2_C4', 2, 'C4', 517.5, 123, 0.35, 0.44, 0.269, 0, 0.384615], [3, 'L_3_C6', 3, 'C6', 450.0, 78, 0.46, 0.38, 0.362, -1, 0.425926], [4, 'L_4_C4', 4, 'C4', 168.75, 124, 0.32, 0.51, 0.276, 2, 0.395058], [5, 'L_5_C6', 5, 'C6', 243.75, 105, 0.49, 0.6, 0.348, 1, 0.550562], [6, 'L_6_C3', 6, 'C3', 150.0, 95, 0.31, 0.56, 0.579, -1, 0.413308], [7, 'L_7_C5', 7, 'C5', 558.75, 108, 0.43, 0.39, 0.795, -1, 0.413462], [8, 'L_8_C6', 8, 'C6', 442.5, 76, 0.38, 0.35, 0.36, -2, 0.368932], [9, 'L_9_C1', 9, 'C1', 300.0, 128, 0.36, 0.35, 0.528, 3, 0.356436], [10, 'L_10_C5', 10, 'C5', 108.75, 77, 0.5, 0.56, 0.806, -2, 0.531915], [11, 'L_11_C1', 11, 'C1', 555.0, 99, 0.31, 0.53, 0.557, -1, 0.397426], [12, 'L_12_C1', 12, 'C1', 412.5, 124, 0.41, 0.33, 0.516, 0, 0.37963], [13, 'L_13_C5', 13, 'C5', 547.5, 88, 0.5, 0.48, 0.811, -1, 0.490196], [14, 'L_14_C2', 14, 'C2', 498.75, 114, 0.44, 0.52, 0.416, -2, 0.478261], [15, 'L_15_C1', 15, 'C1', 101

In [32]:
# Función para predecir el clima con markov
def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
    clima_prediccion = []  # Lista para almacenar la predicción
    tipo_3 = []
    # Inicializar el primer día como seco
    clima_prediccion.append(0)
    tipo_3.append(0)
    # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
    for _ in range(7):
        if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
            clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
            tipo_3.append(0)
        else:  # Si el día anterior fue lluvioso
            clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
            if clima_prediccion[-1] == 1:
                tipo_3.append(1 if random.random() <= 0.05 else 0)
    if sum(tipo_3) >= 1:
        perdida = True
    else:
        perdida = False
    return clima_prediccion, perdida

In [33]:
def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 7
    while index != 0:
        if index == 7:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 6:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad


In [34]:
# funcion que entrega el valor final de la calidad considerando la lluvia
def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

In [35]:
# Parametros para encontrar en la simulacion etapa 1
calidades_lotes = []
lotes_perdidos = []
lotes_obtenidos = []
litros_totales_perdidos = 0
litros_totales_obtenidos = 0
botellas_totales_perdidas = 0
botellas_totales_obtenidas = 0
costos_totales_perdidos = 0
costos_totales_obtenidos = 0

In [36]:
# Simulación de la etapa 1
for t in range(50, 150):
    for lote in lotes:
        lote_id = lote[0]
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_dia_optimo = lote[5]
        lote_prob_seco_lluvia = lote[6]
        lote_prob_lluvia_lluvia = lote[7]
        lote_precio_usd = lote[8]
        lote_variacion = lote[9]
        lote_prob_lluvia = lote[10]
        if t == lote_dia_optimo - 8:
            lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
            if perdida == True:
                lotes_perdidos.append(lote)
                pass
            elif not perdida:
                lista = lote
                if sum(lote_clima_prediccion) > 4:
                    # compra forward
                    lista.append(0)
                elif sum(lote_clima_prediccion) <= 4:
                    # compra spot
                    lista.append(1)
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lista.append(calidad_lote)
                lotes_obtenidos.append(lista)
                
                

In [37]:
for lote in lotes:
    seca_lluvia = lote[6]
    lluvia_lluvia = lote[7]
    porcentaje_de_lluvia = lote[10]

    lluvia_diaria = []
    for dia in range (0,7):
        lluvia_diaria.append(porcentaje_de_lluvia)
    print(lluvia_diaria)
    
    #vemos que 
    C1 = 0.065
    C2 = 0.043
    C3 = 0.065
    C4 = 0.049
    C5 = 0.065
    C6 = 0.043

    #si calculamos como afecta la probabilidad de lluvia por lote según
    

[0.413462, 0.413462, 0.413462, 0.413462, 0.413462, 0.413462, 0.413462]
[0.384615, 0.384615, 0.384615, 0.384615, 0.384615, 0.384615, 0.384615]
[0.425926, 0.425926, 0.425926, 0.425926, 0.425926, 0.425926, 0.425926]
[0.395058, 0.395058, 0.395058, 0.395058, 0.395058, 0.395058, 0.395058]
[0.550562, 0.550562, 0.550562, 0.550562, 0.550562, 0.550562, 0.550562]
[0.413308, 0.413308, 0.413308, 0.413308, 0.413308, 0.413308, 0.413308]
[0.413462, 0.413462, 0.413462, 0.413462, 0.413462, 0.413462, 0.413462]
[0.368932, 0.368932, 0.368932, 0.368932, 0.368932, 0.368932, 0.368932]
[0.356436, 0.356436, 0.356436, 0.356436, 0.356436, 0.356436, 0.356436]
[0.531915, 0.531915, 0.531915, 0.531915, 0.531915, 0.531915, 0.531915]
[0.397426, 0.397426, 0.397426, 0.397426, 0.397426, 0.397426, 0.397426]
[0.37963, 0.37963, 0.37963, 0.37963, 0.37963, 0.37963, 0.37963]
[0.490196, 0.490196, 0.490196, 0.490196, 0.490196, 0.490196, 0.490196]
[0.478261, 0.478261, 0.478261, 0.478261, 0.478261, 0.478261, 0.478261]
[0.373492, 0.

In [38]:
# Modificamos los lotes obtenidos y perdidos con la info relevante, es decir, 
# sacamos las probabilidades de lluvia y el día óptimo de cosecha, entre otras cosas.
lista_final = []
for lote in lotes_obtenidos:
    lista = []
    lote_cod = lote[1]
    lote_numero = lote[2]
    lote_tipo_uva = lote[3]
    lote_toneladas = lote[4]
    lote_costo = lote[8]
    lote_calidad = lote[11]
    lote_contrato = lote[10]
    lote_optimo = lote[5]
    lote_fermentacion = False
    lista.append(lote_cod)
    lista.append(lote_numero)
    lista.append(lote_tipo_uva)
    lista.append(lote_toneladas)
    lista.append(lote_costo)
    lista.append(lote_calidad)
    lista.append(lote_contrato)
    lista.append(lote_optimo)
    lista.append(lote_fermentacion)
    lista_final.append(lista)
print(lotes_obtenidos)
print(lista_final)
print(lotes_perdidos)
lista_final_perdidos = []
for lote in lotes_perdidos:
    lista = []
    lote_cod = lote[1]
    lote_numero = lote[2]
    lote_tipo_uva = lote[3]
    lote_toneladas = lote[4]
    lote_costo = lote[8]
    lote_calidad = 0
    lote_contrato = 1
    lote_optimo = lote[5]
    lista.append(lote_cod)
    lista.append(lote_numero)
    lista.append(lote_tipo_uva)
    lista.append(lote_toneladas)
    lista.append(lote_costo)
    lista.append(lote_calidad)
    lista.append(lote_contrato)
    lista.append(lote_optimo)
    lista_final_perdidos.append(lista)

[[126, 'L_126_C5', 126, 'C5', 187.5, 68, 0.36, 0.35, 0.831, -3, 0.356436, 1, 0.9627488010204082], [284, 'L_284_C3', 284, 'C3', 360.0, 69, 0.49, 0.6, 0.599, -3, 0.550562, 1, 0.9301700685500001], [22, 'L_22_C1', 22, 'C1', 420.0, 71, 0.39, 0.35, 0.554, -1, 0.375, 1, 0.9809329336734693], [115, 'L_115_C3', 115, 'C3', 431.25, 71, 0.44, 0.48, 0.609, -1, 0.458333, 0, 0.92704805441325], [120, 'L_120_C3', 120, 'C3', 217.5, 71, 0.42, 0.46, 0.605, 0, 0.4375, 1, 0.96032425], [194, 'L_194_C4', 194, 'C4', 502.5, 71, 0.35, 0.32, 0.27, -2, 0.339806, 1, 0.9800552521939592], [238, 'L_238_C5', 238, 'C5', 198.75, 72, 0.32, 0.42, 0.808, 1, 0.355556, 1, 0.94295155], [259, 'L_259_C2', 259, 'C2', 127.5, 72, 0.42, 0.32, 0.418, -2, 0.381818, 1, 0.9657024795918367], [266, 'L_266_C6', 266, 'C6', 532.5, 72, 0.45, 0.5, 0.355, -3, 0.473684, 0, 0.9643814860961569], [275, 'L_275_C4', 275, 'C4', 382.5, 72, 0.31, 0.54, 0.276, -1, 0.402584, 1, 0.9969816326530612], [57, 'L_57_C5', 57, 'C5', 232.5, 73, 0.36, 0.52, 0.798, -3

In [39]:
lotes_obtenidos = lista_final
lotes_perdidos = lista_final_perdidos
# Encontrar la info que queramos de esta etapa
print(f'Calidad máxima: {max(calidades_lotes)}')
print(f'Calidad mínima: {min(calidades_lotes)}')
print('')
for lote in lotes_perdidos:
    toneladas = lote[3]
    kilos = toneladas * 1000
    litros = kilos * 0.5
    litros_totales_perdidos += litros
    costo_lote = kilos*lote[4]
    if lote[6] == 0:
        costo_lote = costo_lote * 0.8
        costos_totales_perdidos += costo_lote
    elif lote[6] == 1:
        costo_lote = costo_lote
        costos_totales_perdidos += costo_lote

botellas_totales_perdidas = litros_totales_perdidos * 0.75
print(f'Litros totales perdidos: {round(litros_totales_perdidos)}')
print(f'Botellas totales perdidas: {round(botellas_totales_perdidas)}')
print(f'Costos totales perdidos USD (compra spot): {round(costos_totales_perdidos)}')
print('')
for lote in lotes_obtenidos:
    toneladas = lote[3]
    kilos = toneladas * 1000
    litros = kilos * 0.5
    litros_totales_obtenidos += litros
    costo_lote = kilos*lote[4]
    if lote[6] == 0:
        costo_lote = costo_lote * 0.8
        costos_totales_obtenidos += costo_lote
    elif lote[6] == 1:
        costos_totales_obtenidos += costo_lote*lote[5]
botellas_totales_obtenidas = litros_totales_obtenidos * 0.75
print(f'Litros totales obtenidos: {round(litros_totales_obtenidos)}')
print(f'Botellas totales obtenidas: {round(botellas_totales_obtenidas)}')
print(f'Costos totales esperadas USD (considerando contratos spot y foward): {round(costos_totales_obtenidos)}')
print('')
Costos_totales = costos_totales_obtenidos + costos_totales_perdidos
print(f'Costos totales etapa 1: {round(Costos_totales)}')


Calidad máxima: 1.0
Calidad mínima: 0.8692040782983873

Litros totales perdidos: 2688750
Botellas totales perdidas: 2016562
Costos totales perdidos USD (compra spot): 2566215

Litros totales obtenidos: 45491250
Botellas totales obtenidas: 34118438
Costos totales esperadas USD (considerando contratos spot y foward): 0

Costos totales etapa 1: 2566215


In [40]:
# Guardar los lotes obtenidos en la base de datos
# Ejecutar solo una vez
#import sqlite3



# Conectar a la base de datos (o crearla si no existe)
#conn = sqlite3.connect('etapa_1.db')

### AQUI SACAR # PARA QUE FUNCIONE ESTA CELDA Y SE PUEDA CORRER
#cursor = conn.cursor()

# Crear una tabla
#cursor.execute('''
#CREATE TABLE IF NOT EXISTS lotes (
#    lote_cod TEXT,
#    lote_numero INTEGER,
#    lote_tipo_uva TEXT,
#    lote_toneladas REAL,
#    lote_precio_usd REAL,
#    lote_calidad REAL,
 #   lote_contrato INTEGER
#)
#''')

# Insertar datos en la tabla
#for lote in lista_final:
#    cursor.execute('''
#    INSERT INTO lotes (
#        lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, 
#        lote_precio_usd, lote_calidad, lote_contrato
#    ) VALUES (?, ?, ?, ?, ?, ?, ?)
#    ''', lote)

# Guardar los cambios y cerrar la conexión
#conn.commit()
#conn.close()


In [41]:
### ETAPA 2: CALIDAD DE UVA ###

# Parámetros simulación etapa 2
costo_recuperado = 0
lista_final = lotes_obtenidos + lotes_perdidos
print(lista_final)

[['L_126_C5', 126, 'C5', 187.5, 0.831, 1, 0.356436, 68, False], ['L_284_C3', 284, 'C3', 360.0, 0.599, 1, 0.550562, 69, False], ['L_22_C1', 22, 'C1', 420.0, 0.554, 1, 0.375, 71, False], ['L_115_C3', 115, 'C3', 431.25, 0.609, 0, 0.458333, 71, False], ['L_120_C3', 120, 'C3', 217.5, 0.605, 1, 0.4375, 71, False], ['L_194_C4', 194, 'C4', 502.5, 0.27, 1, 0.339806, 71, False], ['L_238_C5', 238, 'C5', 198.75, 0.808, 1, 0.355556, 72, False], ['L_259_C2', 259, 'C2', 127.5, 0.418, 1, 0.381818, 72, False], ['L_266_C6', 266, 'C6', 532.5, 0.355, 0, 0.473684, 72, False], ['L_275_C4', 275, 'C4', 382.5, 0.276, 1, 0.402584, 72, False], ['L_57_C5', 57, 'C5', 232.5, 0.798, 1, 0.42857, 73, False], ['L_104_C4', 104, 'C4', 487.5, 0.264, 1, 0.356436, 73, False], ['L_20_C4', 20, 'C4', 225.0, 0.274, 1, 0.358696, 74, False], ['L_281_C1', 281, 'C1', 161.25, 0.547, 1, 0.333333, 74, False], ['L_44_C2', 44, 'C2', 367.5, 0.412, 0, 0.523808, 75, False], ['L_55_C2', 55, 'C2', 558.75, 0.408, 1, 0.348315, 75, False], ['L_

In [42]:
contador = 0
for lote in lista_final:
    calidad = lote[5]
    cepa = lote[2]
    precio = lote[4]
    tipo_compra = lote[6]
    kilos = lote[3]*1000
    umbral = umbrales[cepa]
    #Obtenemos costo por lote dependiendo de como ha sido comprado
    if tipo_compra == 0:
        costo = precio * 0.8 * kilos
    else :
        costo = precio * kilos
    if 0.5 <= calidad < umbral:
        recuperado = costo * 0.3 
    elif calidad < 0.5:
        recuperado = costo * 0.05
        # recuperado = costo * calidad * 0.05
    else:
        contador += 1
        recuperado = 0
    costo_recuperado += recuperado
print(f'Costos recuperados etapa 2: {round(costo_recuperado)} USD')

Costos recuperados etapa 2: 463950 USD


### ETAPA 3: FERMENTACIÓN ###

In [43]:
# calcula la duracion en fermentación
def calcular_duracion_vinificacion():
    duracion_aleatoria = random.uniform(7, 9)
    dia_exacto = round(duracion_aleatoria)
    return dia_exacto

In [44]:
# Parámetros simulación etapa 3
costo_plantas = {1:[1600, 1840, 1840], 2:[1500, 1725, 1725], 3:[1800, 2070, 2070]}
uso_plantas = [[0, 0, 0],[0, 0, 0],[0, 0, 0]]
litros_perdidos = []
costos_perdidos = []
dias_tanques = []


In [45]:
def entrar_tanques(litros, costo):
    botellas = 0
    litros_sobrantes = 0
    condicion = False
    while not condicion:
        if litros > 25000:
            litros -= 25000
            botellas += 1
        else:
            litros_sobrantes = litros
            litros_perdidos.append(litros_sobrantes)
            costo_perdido = costo * (litros_sobrantes/0.5)
            costos_perdidos.append(costo_perdido)
            condicion = True
    return botellas

    

In [46]:
def uso_tanques(botellas):
    while botellas > 0:
        if uso_plantas[1][0] < 24:
            uso_plantas[1][0] += 1
            botellas -= 1
        elif uso_plantas[0][0] < 24:
            uso_plantas[0][0] += 1
            botellas -= 1
        elif uso_plantas[1][1] < 24:
            uso_plantas[1][1] += 1
            botellas -= 1
        elif uso_plantas[1][2] < 24:
            uso_plantas[1][2] += 1
            botellas -= 1
        elif uso_plantas[2][0] < 24:
            uso_plantas[2][0] += 1
            botellas -= 1
        elif uso_plantas[0][1] < 24:
            uso_plantas[0][1] += 1
            botellas -= 1
        elif uso_plantas[0][2] < 24:
            uso_plantas[0][2] += 1
            botellas -= 1
        elif uso_plantas[2][1] < 24:
            uso_plantas[2][1] += 1
            botellas -= 1
        elif uso_plantas[2][2] < 24:
            uso_plantas[2][2] += 1
            botellas -= 1
        else:
            break
    return botellas

In [47]:
def sacar_tanques(botellas):
    while botellas > 0:
        if uso_plantas[1][0] > 0:
            uso_plantas[1][0] -= 1
            botellas -= 1
        elif uso_plantas[0][0] > 0:
            uso_plantas[0][0] -= 1
            botellas -= 1
        elif uso_plantas[1][1] > 0:
            uso_plantas[1][1] -= 1
            botellas -= 1
        elif uso_plantas[1][2] > 0:
            uso_plantas[1][2] -= 1
            botellas -= 1
        elif uso_plantas[2][0] > 0:
            uso_plantas[2][0] -= 1
            botellas -= 1
        elif uso_plantas[0][1] > 0:
            uso_plantas[0][1] -= 1
            botellas -= 1
        elif uso_plantas[0][2] > 0:
            uso_plantas[0][2] -= 1
            botellas -= 1
        elif uso_plantas[2][1] > 0:
            uso_plantas[2][1] -= 1
            botellas -= 1
        elif uso_plantas[2][2] > 0:
            uso_plantas[2][2] -= 1
            botellas -= 1
        else:
            break

In [48]:
for t in range(30, 150):
    for lote in lotes_obtenidos:
        lote_cod = lote[0]
        lote_numero = lote[1]
        lote_tipo_uva = lote[2]
        lote_toneladas = lote[3]
        lote_costo = lote[4]
        lote_calidad = lote[5]
        lote_contrato = lote[6]
        lote_dia_cosecha = lote[7]
        lote_salida_fermentacion = lote[8]
        lote_duracion = calcular_duracion_vinificacion()
        if t == lote_dia_cosecha:
            litros_uva = lote_toneladas * 1000 * 0.5
            botellas = entrar_tanques(litros_uva, lote_costo)
            botellas_restantes = uso_tanques(botellas)
            lista = [t + lote_duracion, botellas]
            dias_tanques.append(lista)
            if botellas_restantes > 0:
                print(botellas_restantes)
                costo_perdido = lote_costo * ((botellas_restantes*0.75)/0.5)
                print(costo_perdido)
                costos_perdidos.append(costo_perdido)
            lote[8] = lote_duracion + t
        if lote_salida_fermentacion == t:
            for dias in dias_tanques:
                if dias[0] == t:
                    botella = dias[1]
                    sacar_tanques(botella)
            
                    
            
                 
        
    

2
1.0859999999999999
2
1.209
9
8.235
3
1.5795


In [49]:
perdidas = sum(litros_perdidos)
costos_perdidos = round(sum(costos_perdidos))
print(f'Litros perdidos por no entrar a tanques: {perdidas}')
print(f'Costos perdidos por no entrar a tanques: {costos_perdidos}')


Litros perdidos por no entrar a tanques: 3591250.0
Costos perdidos por no entrar a tanques: 3688742
